#### Dataset Preparation


In [1]:
import numpy as np


np.random.seed(42)

In [2]:
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [3]:
dataset = load_wine()
x = dataset.data
y = dataset.target

In [4]:
df = pd.DataFrame(x, columns=dataset.feature_names)
df["y"] = y
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,y
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


#### CART Classifier


In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [6]:
parameters = {
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 2, 4, 8, 10],
    "min_samples_leaf": [2],
    "max_features": ["sqrt", "log2"],
}

clf = DecisionTreeClassifier()
grid_cv = GridSearchCV(clf, parameters, cv=3)
_ = grid_cv.fit(x_train, y_train)

In [7]:
print(f"Best parameters set found on development set: {grid_cv.best_params_}\n")

means = grid_cv.cv_results_["mean_test_score"]
stds = grid_cv.cv_results_["std_test_score"]

for mean, std, params in zip(means, stds, grid_cv.cv_results_["params"]):
    print(f"{mean:.3f} (+/-{2 * std:.3f}) for {params}")

Best parameters set found on development set: {'criterion': 'entropy', 'max_depth': 4, 'max_features': 'log2', 'min_samples_leaf': 2}

0.887 (+/-0.062) for {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2}
0.872 (+/-0.227) for {'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 2}
0.750 (+/-0.235) for {'criterion': 'gini', 'max_depth': 2, 'max_features': 'sqrt', 'min_samples_leaf': 2}
0.872 (+/-0.157) for {'criterion': 'gini', 'max_depth': 2, 'max_features': 'log2', 'min_samples_leaf': 2}
0.863 (+/-0.058) for {'criterion': 'gini', 'max_depth': 4, 'max_features': 'sqrt', 'min_samples_leaf': 2}
0.904 (+/-0.141) for {'criterion': 'gini', 'max_depth': 4, 'max_features': 'log2', 'min_samples_leaf': 2}
0.823 (+/-0.044) for {'criterion': 'gini', 'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 2}
0.831 (+/-0.067) for {'criterion': 'gini', 'max_depth': 8, 'max_features': 'log2', 'min_samples_leaf': 2}
0.847 (+/-0

#### Train Best Model


In [8]:
clf = DecisionTreeClassifier(**grid_cv.best_params_)
_ = clf.fit(x_train, y_train)

acc = clf.score(x_test, y_test)
print(f"Test Acc: {acc}")

Test Acc: 0.8148148148148148
